In [1]:
import os

import torch

from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image

# Meta params

In [2]:
num_epochs = 100
batch_size = 128
learning_rate = 1e-3

img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Autoencoder definition

In [3]:
class autoencoder(nn.Module):
    
    def __init__(self):
        super(autoencoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True), 
            nn.Linear(64, 12), 
            nn.ReLU(True), 
            nn.Linear(12, 3)
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(True),
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True), 
            nn.Linear(128, 28 * 28), 
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Load data

In [4]:
dataset = MNIST('./data', transform=img_transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')


def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x

# Instantiate the model

In [5]:
model = autoencoder().cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)

# Training

In [6]:
for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img).cuda()
        
        output = model(img)
        loss = criterion(output, img)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss.data.item()))
    if epoch % 5 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, './mlp_img/image_{}.png'.format(epoch))

torch.save(model.state_dict(), './sim_autoencoder.pth')

epoch [1/100], loss:0.1813
epoch [2/100], loss:0.1708
epoch [3/100], loss:0.1607
epoch [4/100], loss:0.1515
epoch [5/100], loss:0.1673
epoch [6/100], loss:0.1451
epoch [7/100], loss:0.1472
epoch [8/100], loss:0.1279
epoch [9/100], loss:0.1555
epoch [10/100], loss:0.1380
epoch [11/100], loss:0.1447
epoch [12/100], loss:0.1430
epoch [13/100], loss:0.1383
epoch [14/100], loss:0.1361
epoch [15/100], loss:0.1340
epoch [16/100], loss:0.1282
epoch [17/100], loss:0.1339
epoch [18/100], loss:0.1391
epoch [19/100], loss:0.1371
epoch [20/100], loss:0.1304
epoch [21/100], loss:0.1200
epoch [22/100], loss:0.1302
epoch [23/100], loss:0.1474
epoch [24/100], loss:0.1308
epoch [25/100], loss:0.1261
epoch [26/100], loss:0.1131
epoch [27/100], loss:0.1351
epoch [28/100], loss:0.1334
epoch [29/100], loss:0.1274
epoch [30/100], loss:0.1231
epoch [31/100], loss:0.1243
epoch [32/100], loss:0.1339
epoch [33/100], loss:0.1475
epoch [34/100], loss:0.1307
epoch [35/100], loss:0.1210
epoch [36/100], loss:0.1433
e